In [1]:
# import team_utils as tu
import pandas as pd
import numpy as np
# import requests
import my_utils2 as mu
from tqdm import tqdm
import json
import datetime


In [2]:
def rawdataLoader(datas: str):
    return json.loads(datas.replace("'", '"').replace('True', '1').replace('False', '0'))

In [17]:
def get_champion_spell():
    now = datetime.datetime.now()
    now = now.strftime('%Y-%m-%d %H:%M:%S')
    
    df = mu.oracle_totalExecute("SELECT version,game_id, champion_id ,participant_number, champion, spell FROM RAWDATA")
    
    df = df.sort_values(['GAME_ID','PARTICIPANT_NUMBER'])
    df_cols = df.columns[4:]
    
    matches = {a:[] for a in df_cols}
    for rec in tqdm(df.iloc):
        for col in [a for a in df_cols]:
            matches[col].append(rawdataLoader(rec[col]))
    for col in [a for a in df_cols]:
        df[col] = matches[col]
    
    champid_lst = df['CHAMPION_ID'].tolist()
    champion_list =  df['CHAMPION'].tolist()
    spell_list =  df['SPELL'].tolist()
    version_list = df['VERSION'].tolist()
    
    print('데이터 추출....')
    
    result=[]
    col = ['version','update_time','championId','lane','win','summoner1Id','summoner2Id']
    for idx,champ in enumerate(tqdm(champion_list)):
        result.append([
            version_list[idx],
            now,
            champid_lst[idx],
            champ['teamPosition'],
            int(champ['win']),
            spell_list[idx]['summoner1Id'],
            spell_list[idx]['summoner2Id'],
        ])
    champ_spell = pd.DataFrame(result,columns=col)
    
    spell_lst = champ_spell[['summoner1Id','summoner2Id']].values.tolist()
    
    result_lst=[]
    for i in spell_lst:
         result_lst.append(sorted(i))
    
    df_lst = pd.DataFrame(result_lst,columns=['s1_id','s2_id'])
    
    result_df = champ_spell.join(df_lst)
    print('데이터 그룹화')
    
    
    #승률 그룹바이
    result_df_count = result_df.groupby(['championId','lane','s1_id','s2_id']).count()[['win']].rename(columns={'win':'count'})
    result_df_win = result_df.groupby(['championId','lane','s1_id','s2_id']).sum()[['win']].rename(columns={'win':'total_win'})
    
    result_df_count_win = result_df_count.join(result_df_win)
    
    result_df_count_win.reset_index(inplace=True)
    result_df_count_win['win_rate']= result_df_count_win['total_win']/result_df_count_win['count']
    result_df_count_win['pick_rate']= result_df_count_win['count']/sum(result_df_count_win['count'])
    
    return result_df_count_win

In [18]:
df1= get_champion_spell()

oracle open!
oracle close!


364900it [00:55, 6627.08it/s]


데이터 추출....


100%|██████████████████████████████████████████████████████████████████████| 364900/364900 [00:01<00:00, 292099.40it/s]


데이터 그룹화


In [19]:
df1

,championId,lane,s1_id,s2_id,count,total_win,win_rate,pick_rate
0,1,BOTTOM,4,6,2,1,0.500000,0.000005
1,1,BOTTOM,4,7,1,0,0.000000,0.000003
2,1,BOTTOM,4,12,1,1,1.000000,0.000003
3,1,BOTTOM,4,14,1,1,1.000000,0.000003
4,1,MIDDLE,1,4,5,2,0.400000,0.000014
...,...,...,...,...,...,...,...,...
2905,902,UTILITY,4,7,2298,1184,0.515231,0.006298
2906,902,UTILITY,4,12,2,1,0.500000,0.000005
2907,902,UTILITY,4,14,499,258,0.517034,0.001367
2908,902,UTILITY,4,21,2,2,1.000000,0.000005


In [20]:
df1.rename(columns={'s1_id':'spell1','s2_id':'spell2'},inplace=True)

In [21]:
df1

,championId,lane,spell1,spell2,count,total_win,win_rate,pick_rate
0,1,BOTTOM,4,6,2,1,0.500000,0.000005
1,1,BOTTOM,4,7,1,0,0.000000,0.000003
2,1,BOTTOM,4,12,1,1,1.000000,0.000003
3,1,BOTTOM,4,14,1,1,1.000000,0.000003
4,1,MIDDLE,1,4,5,2,0.400000,0.000014
...,...,...,...,...,...,...,...,...
2905,902,UTILITY,4,7,2298,1184,0.515231,0.006298
2906,902,UTILITY,4,12,2,1,0.500000,0.000005
2907,902,UTILITY,4,14,499,258,0.517034,0.001367
2908,902,UTILITY,4,21,2,2,1.000000,0.000005


In [10]:
spell_lst = df1[['summoner1Id','summoner2Id']].values.tolist()

In [11]:
spell_lst

[[12, 4],
 [4, 11],
 [14, 4],
 [3, 4],
 [4, 14],
 [12, 4],
 [11, 4],
 [12, 4],
 [4, 7],
 [3, 4],
 [4, 14],
 [11, 4],
 [1, 6],
 [7, 4],
 [14, 4],
 [12, 4],
 [11, 4],
 [4, 14],
 [4, 7],
 [14, 4],
 [4, 12],
 [11, 4],
 [14, 12],
 [4, 7],
 [14, 4],
 [12, 4],
 [4, 11],
 [14, 4],
 [12, 4],
 [4, 14],
 [4, 12],
 [4, 11],
 [12, 4],
 [7, 4],
 [4, 3],
 [14, 4],
 [11, 4],
 [4, 14],
 [3, 4],
 [4, 14],
 [4, 12],
 [11, 4],
 [4, 14],
 [7, 4],
 [14, 4],
 [12, 4],
 [4, 11],
 [14, 4],
 [3, 4],
 [21, 4],
 [4, 12],
 [4, 11],
 [12, 4],
 [1, 4],
 [14, 4],
 [4, 14],
 [4, 11],
 [12, 4],
 [1, 4],
 [4, 3],
 [4, 12],
 [4, 11],
 [4, 14],
 [7, 4],
 [4, 14],
 [12, 4],
 [11, 4],
 [4, 12],
 [4, 7],
 [4, 14],
 [14, 12],
 [6, 11],
 [12, 4],
 [7, 4],
 [4, 14],
 [12, 4],
 [11, 4],
 [14, 4],
 [7, 4],
 [14, 4],
 [14, 4],
 [11, 4],
 [14, 4],
 [4, 7],
 [14, 4],
 [12, 14],
 [6, 11],
 [4, 14],
 [7, 4],
 [14, 4],
 [4, 14],
 [11, 4],
 [14, 4],
 [7, 4],
 [3, 4],
 [4, 12],
 [4, 11],
 [14, 4],
 [7, 4],
 [3, 4],
 [12, 4],
 [11, 4],
 [

In [13]:
result_lst=[]
for i in spell_lst:
     result_lst.append(sorted(i))
    
df_lst = pd.DataFrame(result_lst,columns=['s1_id','s2_id'])

In [15]:
result_lst

[[4, 12],
 [4, 11],
 [4, 14],
 [3, 4],
 [4, 14],
 [4, 12],
 [4, 11],
 [4, 12],
 [4, 7],
 [3, 4],
 [4, 14],
 [4, 11],
 [1, 6],
 [4, 7],
 [4, 14],
 [4, 12],
 [4, 11],
 [4, 14],
 [4, 7],
 [4, 14],
 [4, 12],
 [4, 11],
 [12, 14],
 [4, 7],
 [4, 14],
 [4, 12],
 [4, 11],
 [4, 14],
 [4, 12],
 [4, 14],
 [4, 12],
 [4, 11],
 [4, 12],
 [4, 7],
 [3, 4],
 [4, 14],
 [4, 11],
 [4, 14],
 [3, 4],
 [4, 14],
 [4, 12],
 [4, 11],
 [4, 14],
 [4, 7],
 [4, 14],
 [4, 12],
 [4, 11],
 [4, 14],
 [3, 4],
 [4, 21],
 [4, 12],
 [4, 11],
 [4, 12],
 [1, 4],
 [4, 14],
 [4, 14],
 [4, 11],
 [4, 12],
 [1, 4],
 [3, 4],
 [4, 12],
 [4, 11],
 [4, 14],
 [4, 7],
 [4, 14],
 [4, 12],
 [4, 11],
 [4, 12],
 [4, 7],
 [4, 14],
 [12, 14],
 [6, 11],
 [4, 12],
 [4, 7],
 [4, 14],
 [4, 12],
 [4, 11],
 [4, 14],
 [4, 7],
 [4, 14],
 [4, 14],
 [4, 11],
 [4, 14],
 [4, 7],
 [4, 14],
 [12, 14],
 [6, 11],
 [4, 14],
 [4, 7],
 [4, 14],
 [4, 14],
 [4, 11],
 [4, 14],
 [4, 7],
 [3, 4],
 [4, 12],
 [4, 11],
 [4, 14],
 [4, 7],
 [3, 4],
 [4, 12],
 [4, 11],
 [

In [14]:
df_lst

,s1_id,s2_id
0,4,12
1,4,11
2,4,14
3,3,4
4,4,14
...,...,...
364895,4,6
364896,4,11
364897,4,14
364898,4,7


In [16]:
 result_df = df1.join(df_lst)

In [17]:
result_df

,version,update_time,championId,lane,win,summoner1Id,summoner2Id,s1_id,s2_id
0,12.5.425.9171,2023-05-25 13:17:16,126,TOP,0,12,4,4,12
1,12.5.425.9171,2023-05-25 13:17:16,104,JUNGLE,0,4,11,4,11
2,12.5.425.9171,2023-05-25 13:17:16,166,MIDDLE,0,14,4,4,14
3,12.5.425.9171,2023-05-25 13:17:16,145,BOTTOM,0,3,4,3,4
4,12.5.425.9171,2023-05-25 13:17:16,12,UTILITY,0,4,14,4,14
...,...,...,...,...,...,...,...,...,...
364895,13.10.509.8402,2023-05-25 13:17:16,122,TOP,0,6,4,4,6
364896,13.10.509.8402,2023-05-25 13:17:16,121,JUNGLE,0,4,11,4,11
364897,13.10.509.8402,2023-05-25 13:17:16,245,MIDDLE,0,14,4,4,14
364898,13.10.509.8402,2023-05-25 13:17:16,498,BOTTOM,0,7,4,4,7


In [46]:
#승률 그룹바이
result_df_count = result_df.groupby(['lane','s1_id','s2_id']).count()[['win']].rename(columns={'win':'count'})
result_df_win = result_df.groupby(['lane','s1_id','s2_id']).sum()[['win']].rename(columns={'win':'total_win'})

In [47]:
result_df_count.sort_values('count',ascending=False)

count
lane    s1_id s2_id       
JUNGLE  4     11     66725
UTILITY 4     14     47971
TOP     4     12     47513
MIDDLE  4     14     32849
              12     32544
...                    ...
BOTTOM  14    21         1
JUNGLE  6     12         1
MIDDLE  11    12         1
JUNGLE  3     14         1
MIDDLE  12    21         1

[119 rows x 1 columns]

In [21]:
result_df_win

total_win
lane    s1_id s2_id           
BOTTOM  1     4           6952
              6             37
              7              1
              12             0
              14             1
...                        ...
UTILITY 7     21             0
        11    12             0
              14             2
        12    14             6
        14    21             8

[119 rows x 1 columns]

In [28]:
result_df_count_win = result_df_count.join(result_df_win)

In [29]:
result_df_count_win

count  total_win
lane    s1_id s2_id                  
BOTTOM  1     4      14077       6952
              6         76         37
              7          1          1
              12         1          0
              14         1          1
...                    ...        ...
UTILITY 7     21         1          0
        11    12         2          0
              14         3          2
        12    14        12          6
        14    21        14          8

[119 rows x 2 columns]

In [30]:
result_df_count_win.reset_index(inplace=True)
result_df_count_win['win_rate']= result_df_count_win['total_win']/result_df_count_win['count']
result_df_count_win['pick_rate']= result_df_count_win['count']/sum(result_df_count_win['count'])

In [ ]:
result_df_count_win['win_rate']= result_df_count_win['total_win']/result_df_count_win['count']

In [32]:
result_df_count_win['pick_rate']= result_df_count_win['count']/sum(result_df_count_win['count'])

In [33]:
result_df_count_win

,lane,s1_id,s2_id,count,total_win,win_rate,pick_rate
0,BOTTOM,1,4,14077,6952,0.493855,0.038578
1,BOTTOM,1,6,76,37,0.486842,0.000208
2,BOTTOM,1,7,1,1,1.000000,0.000003
3,BOTTOM,1,12,1,0,0.000000,0.000003
4,BOTTOM,1,14,1,1,1.000000,0.000003
...,...,...,...,...,...,...,...
114,UTILITY,7,21,1,0,0.000000,0.000003
115,UTILITY,11,12,2,0,0.000000,0.000005
116,UTILITY,11,14,3,2,0.666667,0.000008
117,UTILITY,12,14,12,6,0.500000,0.000033
